In [1]:
from tum_dlr_automl_for_eo.datamodules.EODataLoader import EODataModule
import matplotlib.pyplot as plt 
import numpy as np 
import torch
import torchvision 
import torchvision.transforms as transforms 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

In [ ]:
data_module = EODataModule('./', 'Sentinel-2')
data_module.prepare_data()
transform = transforms.Compose([transforms.ToTensor()])
data_module.setup(transform)
training_data  = data_module.training_dataLoader(batch_size = 10000)

In [3]:
testing_data  = data_module.testing_dataLoader(batch_size = 1000)

In [4]:
class Net(nn.Module):
    ''' Models a simple Convolutional Neural Network'''
    def __init__(self):
        
        super(Net, self).__init__()
    # 3 input image channel, 6 output channels, 
    # 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(10, 6, 5)
    # Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)# 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 17)

    def forward(self, x):
        
    
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(10, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=17, bias=True)
)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)
print(len(training_data))

353


In [6]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(training_data,0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0], data[1]
       

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 2.371
[1,   200] loss: 1.880
[1,   300] loss: 1.702
[2,   100] loss: 1.564
[2,   200] loss: 1.500
[2,   300] loss: 1.530
[3,   100] loss: 1.382
[3,   200] loss: 1.317
[3,   300] loss: 1.278
[4,   100] loss: 1.284
[4,   200] loss: 1.236
[4,   300] loss: 1.204
[5,   100] loss: 1.170
[5,   200] loss: 1.106
[5,   300] loss: 1.131
